# Baryon-Baryon Correlator

First we import stuff, and define all of the objects needed to create/annihilate baryons.  In the future this will be read in from a text file that is generated by some group theory code to construct operators in an automated way.  

In [1]:
import sys
import os

sys.path.append(os.getcwd()[:-9])

from WickContractions.ops.operator import *
from WickContractions.ops.elemental import *
from WickContractions.ops.quarks import *
from WickContractions.ops.commuting import *
from WickContractions.corrs.diagram import *
from WickContractions.wick.utilities import *
from WickContractions.wick.contract import *
from WickContractions.laph.diagram import *

from IPython.display import display, Math

In [2]:
c0=EpsilonTensor(['c_{0}','c_{1}','c_{2}','c_{3}'])
c1=SpinMatrix('\\Gamma^2',['s_{0}','s_{1}','s_{2}','s_{3}'])
                  
q0=Quark(False,'u','s_{0}','c_{0}','t_f','x_1')
q1=Quark(False,'d','s_{1}','c_{1}','t_f','x_1')
#q1=Quark(False,'d','s1','c1','tf','x1')
q2=Quark(False,'u','s_{2}','c_{2}','t_f','x_1')
#q2=Quark(False,'u','s2','c2','tf','x1')
q3=Quark(False,'d','s_{3}','c_{3}','t_f','x_1')


annihilate_baryon = Operator([ElementalOperator(1,[c0,c1],[q0,q1,q2,q3])])

c6=EpsilonTensor(['c_{4}','c_{5}','c_{6}','c_{7}'])
c7=SpinMatrix('\\Gamma^1',['s_{4}','s_{5}','s_{6}','s_{7}'])

q6=Quark(True,'d','s_{4}','c_{4}','t_i','x_2')
#q6=Quark(True,'d','s4','c4','ti','x2')
q7=Quark(True,'u','s_{5}','c_{5}','t_i','x_2')
q8=Quark(True,'d','s_{6}','c_{6}','t_i','x_2')
#q9=Quark(True,'u','s7','c7','ti','x2')
q9=Quark(True,'u','s_{7}','c_{7}','t_i','x_2')

create_baryon = Operator([ElementalOperator(1,[c6,c7],[q6,q7,q8,q9])])

In [3]:
display(Math(str(annihilate_baryon)))

<IPython.core.display.Math object>

In [4]:
display(Math(str(create_baryon)))

<IPython.core.display.Math object>

Now do the contractions and store in Laph subspace

In [5]:
res = contract(annihilate_baryon, create_baryon).diagrams

print("B->B correlator has {} diagrams".format(len(res)))

# below is very verbose output
#for d in res:
#    print(d)

# needs to be run on position space correlators
for i in range(len(res)):
    res[i]=LDiagram(res[i])

B->B correlator has 4 diagrams


Now we can apply laph smearing to the quarks.  We print out the B->B diagrams explicitly since there are only four. 

In [6]:


#testing a more generic way to make blocks.
tst=copy.deepcopy(res)
for d in tst:
    d.create_baryon_blocks()
    #print(d)
    print(d.Tblocks)
    print(d.Bblocks)
    #print()
    #d.create_block('delta','M') # should allow this code to handle any number of mesons/baryons
    d.combine_indices()
    display(Math(str(d)))

print()

#for d in tst:
    #print(d)
    #d.create_baryon_source()       # not NC agnostic at the moment.
                                   # not generalized to accomodate mesons + baryons
    #print(d)
    #print()

{'T(x_2,t_i)_{l_{4} l_{5} l_{6} l_{7}}': '\\epsilon_{c_{4} c_{5} c_{6} c_{7}}V(x_2,t_i)_{c_{4} l_{4}}V(x_2,t_i)_{c_{5} l_{5}}V(x_2,t_i)_{c_{6} l_{6}}V(x_2,t_i)_{c_{7} l_{7}}', 'T^*(x_1,t_f)_{l_{0} l_{1} l_{2} l_{3}}': '\\epsilon_{c_{0} c_{1} c_{2} c_{3}}V*(x_1,t_f)_{c_{0} l_{0}}V*(x_1,t_f)_{c_{1} l_{1}}V*(x_1,t_f)_{c_{2} l_{2}}V*(x_1,t_f)_{c_{3} l_{3}}'}
{'B^*(x_1,t_f,\\Gamma^2)_{l_{0} l_{1} l_{2} l_{3} s_{0} s_{1} s_{2} s_{3}}': 'T^*(x_1,t_f)_{l_{0} l_{1} l_{2} l_{3}}\\Gamma^2_{s_{0} s_{1} s_{2} s_{3}}', 'B(x_2,t_i,\\Gamma^1)_{l_{4} l_{5} l_{6} l_{7} s_{4} s_{5} s_{6} s_{7}}': 'T(x_2,t_i)_{l_{4} l_{5} l_{6} l_{7}}\\Gamma^1_{s_{4} s_{5} s_{6} s_{7}}'}


<IPython.core.display.Math object>

{'T(x_2,t_i)_{l_{4} l_{5} l_{6} l_{7}}': '\\epsilon_{c_{4} c_{5} c_{6} c_{7}}V(x_2,t_i)_{c_{4} l_{4}}V(x_2,t_i)_{c_{5} l_{5}}V(x_2,t_i)_{c_{6} l_{6}}V(x_2,t_i)_{c_{7} l_{7}}', 'T^*(x_1,t_f)_{l_{0} l_{1} l_{2} l_{3}}': '\\epsilon_{c_{0} c_{1} c_{2} c_{3}}V*(x_1,t_f)_{c_{0} l_{0}}V*(x_1,t_f)_{c_{1} l_{1}}V*(x_1,t_f)_{c_{2} l_{2}}V*(x_1,t_f)_{c_{3} l_{3}}'}
{'B^*(x_1,t_f,\\Gamma^2)_{l_{0} l_{1} l_{2} l_{3} s_{0} s_{1} s_{2} s_{3}}': 'T^*(x_1,t_f)_{l_{0} l_{1} l_{2} l_{3}}\\Gamma^2_{s_{0} s_{1} s_{2} s_{3}}', 'B(x_2,t_i,\\Gamma^1)_{l_{4} l_{5} l_{6} l_{7} s_{4} s_{5} s_{6} s_{7}}': 'T(x_2,t_i)_{l_{4} l_{5} l_{6} l_{7}}\\Gamma^1_{s_{4} s_{5} s_{6} s_{7}}'}


<IPython.core.display.Math object>

{'T(x_2,t_i)_{l_{4} l_{5} l_{6} l_{7}}': '\\epsilon_{c_{4} c_{5} c_{6} c_{7}}V(x_2,t_i)_{c_{4} l_{4}}V(x_2,t_i)_{c_{5} l_{5}}V(x_2,t_i)_{c_{6} l_{6}}V(x_2,t_i)_{c_{7} l_{7}}', 'T^*(x_1,t_f)_{l_{0} l_{1} l_{2} l_{3}}': '\\epsilon_{c_{0} c_{1} c_{2} c_{3}}V*(x_1,t_f)_{c_{0} l_{0}}V*(x_1,t_f)_{c_{1} l_{1}}V*(x_1,t_f)_{c_{2} l_{2}}V*(x_1,t_f)_{c_{3} l_{3}}'}
{'B^*(x_1,t_f,\\Gamma^2)_{l_{0} l_{1} l_{2} l_{3} s_{0} s_{1} s_{2} s_{3}}': 'T^*(x_1,t_f)_{l_{0} l_{1} l_{2} l_{3}}\\Gamma^2_{s_{0} s_{1} s_{2} s_{3}}', 'B(x_2,t_i,\\Gamma^1)_{l_{4} l_{5} l_{6} l_{7} s_{4} s_{5} s_{6} s_{7}}': 'T(x_2,t_i)_{l_{4} l_{5} l_{6} l_{7}}\\Gamma^1_{s_{4} s_{5} s_{6} s_{7}}'}


<IPython.core.display.Math object>

{'T(x_2,t_i)_{l_{4} l_{5} l_{6} l_{7}}': '\\epsilon_{c_{4} c_{5} c_{6} c_{7}}V(x_2,t_i)_{c_{4} l_{4}}V(x_2,t_i)_{c_{5} l_{5}}V(x_2,t_i)_{c_{6} l_{6}}V(x_2,t_i)_{c_{7} l_{7}}', 'T^*(x_1,t_f)_{l_{0} l_{1} l_{2} l_{3}}': '\\epsilon_{c_{0} c_{1} c_{2} c_{3}}V*(x_1,t_f)_{c_{0} l_{0}}V*(x_1,t_f)_{c_{1} l_{1}}V*(x_1,t_f)_{c_{2} l_{2}}V*(x_1,t_f)_{c_{3} l_{3}}'}
{'B^*(x_1,t_f,\\Gamma^2)_{l_{0} l_{1} l_{2} l_{3} s_{0} s_{1} s_{2} s_{3}}': 'T^*(x_1,t_f)_{l_{0} l_{1} l_{2} l_{3}}\\Gamma^2_{s_{0} s_{1} s_{2} s_{3}}', 'B(x_2,t_i,\\Gamma^1)_{l_{4} l_{5} l_{6} l_{7} s_{4} s_{5} s_{6} s_{7}}': 'T(x_2,t_i)_{l_{4} l_{5} l_{6} l_{7}}\\Gamma^1_{s_{4} s_{5} s_{6} s_{7}}'}


<IPython.core.display.Math object>

### Quick Check of computation

I just did a quick check of the loop cost and printed out each contraction that needed to be done.  The latter notation is used in Eigen::Tensor or Fastor.

In [7]:
#brute force counting.
costs = {}
for d in tst:
    nested_sums = 0
    for idx in d.commuting[0].indices[0:4]:
        if(idx in d.commuting[1].indices):
            nested_sums+=1
    if nested_sums in costs:
        costs[nested_sums]+=1
    else:
        costs[nested_sums]=1
print('total loops:')
for k,v in costs.items():
    print('  {}*N^{}'.format(v,k))

total loops:
  4*N^0


In [8]:
for d in tst:
    contractions = []
    for i,idx in enumerate(d.commuting[0].indices[0:4]):
        j=d.commuting[1].indices.index(idx)
        contractions.append([i,j])
    print(contractions)

ValueError: '{0}' is not in list

## Diagram Plots

Just making some quick visuals of the topologies.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import copy

tst=copy.deepcopy(res)
for d in tst:
    d.create_baryon_blocks()
    d.short_props()

for d in tst:
    d.create_baryon_source()
    #print(d)
    
G = nx.Graph()
G.add_node('4ti',pos=(1,1))
G.add_node('5ti',pos=(1,2))
G.add_node('6ti',pos=(1,3))
G.add_node('7ti',pos=(1,4))
G.add_node('4tf',pos=(5,1))
G.add_node('5tf',pos=(5,2))
G.add_node('6tf',pos=(5,3))
G.add_node('7tf',pos=(5,4))
pos=nx.get_node_attributes(G,'pos')

xdim = int(len(tst)/2)
ydim = int(len(tst)-len(tst)/2)
for iplt,d in enumerate(tst):
    Gtmp=G.copy()
    pos=nx.get_node_attributes(G,'pos')

    for i,idx in enumerate(d.commuting[0].indices):
        Gtmp.add_edge(idx+d.commuting[0].arguments[1],
                   d.commuting[1].indices[i]+tst[0].commuting[0].arguments[2])
    plt.subplot(xdim,ydim,iplt+1)
    nx.draw(Gtmp,pos)
#plt.savefig('baryon_correlator.png')
plt.show()